<div style="
    display: flex;
    justify-content: center;
    align-items: center;
    height: 100px;
    border: 4px solid #32CD32;
    border-radius: 15px;
    background-color: #000000;
    box-shadow: 0 4px 8px 0 rgba(255, 215, 0, 0.2);
    margin: 20px;
">
    <h1 style="
        color: #32CD32;
        font-family: 'Nexa','', ;
        font-size: 52px;
        text-shadow: 2px 2px 4px #32CD32;
    ">
        MASSIVEChem 
    </h1>
</div>

## Introduction
The following Jupyter notebook briefly presents MASSIVEChem, a pip-installable python package. This package is developped around mass spectrometry, a tool commonly used in analytical chemistry, biochemistry, pharmaceutical sciences and more. Fundamentally, this package simulates and plots the mass spectrum of a given molecule, with a chosen resolution and with a chosen precision. 

This package was created as a collaborative project for the EPFL course Practical programming in chemistry [![GitHub3](https://img.shields.io/badge/EPFL-CH200-red.svg)](https://edu.epfl.ch/studyplan/en/bachelor/chemistry-and-chemical-engineering/coursebook/practical-programming-in-chemistry-CH-200)

Before going through the code and the use of the package, let's briefly discuss the ideas that allowed to build this package

## Ideas

Initially, the plan for this project was to write a code that would predict the NMR spectrum of an input molecule. 3 weeks were spent trying to figure out a way to analyze molecules, to find functional groups, and to predict the shift of the hydrogens in a molecule. As the time was advancing, it became apparent to us that this project was beyond the scope of our abilities as it would most likely require the use of machine learning, a concept that none of us was familiar with. 

Still wanting to create a package that would display some type of visual information from a molecule, our minds collectively turned towards mass spectroscopy. Being familiar with the theoretical concepts behind this analytical method, having followed a course on mass spectroscopy in first year and having revisited the topic in our second-year electromagnetics course, we were more confident in our abilities regarding this idea.

The concept of mass spectrometry consists of ionizing a molecule and passing this molecule in a strong magnetic field. Thanks to Lorentz' force, the molecule is deviated as a function of its mass which allows to separate the different isotopes present in a sample. By  putting a detector at the end, it is possible to determine which molecule was put in the mass spectrometer.

![Alt text](Schematic_mass_spectrum.png)

Coding wise, simulating the mass spectrum was done by using list comprehension and combinatory probabilities. This involves a systematic breakdown of the process into discrete steps

- Molecule Definition: First the molecules to be analyzed needs to be defined, specifying its masses and relative abundances. Each molecule can be represented as a data structure (a list) containing its mass and abundance.

- Fragmentation Simulation: Then, the fragmentation process needs to be simulated. This step involves generating all possible combinations of masses for each atom using the algorithm of probability trees. 

- Mass Calculation: The Mass of each fragment is calculated by summing the masses of the constituent atoms. This step ensures that each fragment is assigned an accurate mass value based on its atomic composition.

- Combination of Fragments: At the end of the mass calculation process, all fragments are combined into a single list and the fragment with similar masses are combined. This aggregated list represents the collective set of fragments resulting from the analyzed molecules.

- Mass Spectrum Simulation: Simulate the mass spectrum by estimating the abundance of each fragment. This involves considering both the relative abundances of the original molecules and the probabilities associated with generating each fragment. List comprehension facilitates the efficient calculation of fragment abundances across the entire spectrum.

- Visualization: Visualize the simulated mass spectrum using appropriate plotting techniques. Tools like Matplotlib in Python enable the creation of detailed plots that illustrate the distribution of fragment masses and their corresponding abundances.

By following this structured approach and leveraging the power of list comprehensions and combinatory probabilities, it becomes possible to create accurate simulations of mass spectrometry experiments, offering insights into the composition and characteristics of the analyzed molecules.


## Difficulties
Initially, the plan for this project was to write a code that would predict the NMR spectrum of an input molecule. However, it quickly became apparent to us that this project was beyond the scope of our abilities as it would most likely require the use of machine learning, a concept that none of us was familiar with.

Still wanting to create a package that would display some type of visual information from a molecule, our minds collectively turned towards mass spectroscopy. Being familiar with the theoretical concepts behind this analytical method, having followed a course on mass spectroscopy in first year and having revisited the topic in our second-year electromagnetics course, we were more confident in our abilities regarding this idea.

Once we had written the code that would do the actual computation of the different combinations of isotopes that can compose the molecule (with their relative probabilities of existing), it became very noticeable that the time complexity of the algorithm was subpar (being of roughly 2^n with n being the number of atoms in the molecule; the code would take ~30s to run for pentane). To remedy this fact, a part of code was implemented that would disregard any molecule with a probability of under 10^-6 of existing (this reduced the time complexity form 2^n to n).


## Limitations



To begin using it, simply type the following command into the terminal:

In [5]:
!pip install MASSIVEChem

Once this is done, a whole set of functions will be imported.
For example, to use the function that takes a molecule under SMILEs representation, a precision boolean and an apparatus resolution as inputs, and outputs the spectrum, one has to do the following:

In [10]:
import MASSiveChem.MASSiveChem as MC
from bokeh.plotting import show, output_notebook
output_notebook()
show(MC.spectrum('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C', True, 0.01))

Loading BokehJS ...

FileNotFoundError: [Errno 2] No such file or directory: 'data/abundance.txt'

Note: the second argument here being True indicates that the function will neglect any ions which have a probability of apparation under 0.00001

## Examples of functions and their respective inputs and outputs
First, here is a list of the functions that come with the package: 

## Basic functions
-calculate_unsaturation(mol_smi)->unsaturation

-SMILEs_interpreter(mol_smi)->mol

-molecule_list_generator(mol)->list_atoms

-ionisation_method(list_atoms)->list_atoms

-sulphur_nitrogen_finder(list_atoms)->has_N, has_S, count_N, count_S

-main_function(list_atoms, imprecision_True_false)->list_masses & list_abundances

-peak_merger(list_ions, list_abundances)->list_masses & list_abundances

-sulphur_nitrogen_adder(list_ions, list_abundances, has_N, has_S, count_N, count_S)->list_masses, list_abundances

-peak_sorter(list_masses, list_abundances)->list_masses & list_abundances

-delta_function_plotter(list_masses, list_abundances)->list_masses & list_abundances

-double_plot(list_masses, list_abundances)->bokeh_plot_1, bokeh_plot_2

-functional_group_finder(mol_smi)->list_functionalgroups_contained

-functional_group_display(ist_functionalgroups_contained)-> bokeh_plot_3

-mol_web_show(mol_smi)->bokeh_plot_4

-all_in_one(bokeh_plot_1, bokeh_plot_2, bokeh_plot_3, bokeh_plot_4)

-smiles_to_3D_plot(mol_smi)-> XXX

from all of these functions, the following two main functions are most useful:
## Main functions



-spectrum(mol_smi, imprecision_True_False, apparatus_resolution)

-specturm_3D(mol_smi, imprecision_True_False, apparatus_resolution)

## Functionality
The main functions can both be made by combining the basic functions.

## Examples
First, all of the basic functions will be shown, in order to then construct spectrum and spectrum_3D out of them.



## SMILEs interpreter
This function takes a molecule under SMILEs representation as input and output its MOL represerntation

In [1]:
import MASSiveChem.MASSiveChem as MC
print(MC.SMILEs_interpreter('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C'))

## molecule_list_generator
This function takes a MOL representation as input and outputs a list of all of the atoms within that molecule

In [2]:
import MASSiveChem.MASSiveChem as MC
mol = MC.SMILEs_interpreter('CC1(C(N2C(S1)C(C2=O)NC(=O)CC3=CC=CC=C3)C(=O)O)C')
print(MC.molecule_list_generator(mol))

['C', 'C', 'C', 'N', 'C', 'S', 'C', 'C', 'O', 'N', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'O', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']


## ionisation_method(list_atoms)
This function takes a list of atoms as input and simply outputs the same list with a proton less (in essence, a hydrogen atom)

In [3]:
import MASSiveChem.MASSiveChem as MC

list_atoms = ['C', 'C', 'C', 'N', 'C', 'S', 'C',
            'C', 'O', 'N', 'C', 'O', 'C', 'C',
            'C', 'C', 'C', 'C', 'C', 'C', 'O',
            'O', 'C', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H']

print(MC.ionisation_method(list_atoms))

['C', 'C', 'C', 'N', 'C', 'S', 'C', 'C', 'O', 'N', 'C', 'O', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'O', 'O', 'C', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H']


## sulphur_nitrogen_finder(list_atoms)
This function takes a list of atoms as input and output four values: has_N, has_S, count_N and count_S.
has_N & has_S are True if there is an odd number of Nitrogen and Sulphur repsectively, and False otherwise.

In [4]:
import MASSiveChem.MASSiveChem as MC

list_atoms = ['C', 'C', 'C', 'N', 'C', 'S', 'C',
            'C', 'O', 'N', 'C', 'O', 'C', 'C',
            'C', 'C', 'C', 'C', 'C', 'C', 'O',
            'O', 'C', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H', 'H', 'H',
            'H', 'H', 'H', 'H', 'H']

print(MC.sulphur_nitrogen_finder(list_atoms))

AttributeError: module 'MASSiveChem.MASSiveChem' has no attribute 'sulphur_nitrogen_finder'

## main_function(list_atoms)
This function takes a list of atoms as an input, and will output two lists.

The first (list_masses) is a list of the masses of all of the possible combinations of isotopes that can form the given molecule (under list representation) and the second (list_abundances) is a list of the probabilities of apparition of each of these combinations of isotopes.

As such, both lists are the same length and the mass at index i in list_masses has the probability at index i in list_abundances associated to it.

In [6]:
import MASSiveChem.MASSiveChem as MC

output = MC.main_function(['C', 'C', 'C', 'N', 'C', 'S', 'C','C', 'O', 'N', 'C', 'O', 'C', 'C',
                            'C', 'C', 'C', 'C', 'C', 'C', 'O','O', 'C', 'H', 'H', 'H', 'H', 'H',
                            'H', 'H', 'H', 'H', 'H', 'H', 'H','H', 'H', 'H', 'H', 'H'], True)

print(output)

FileNotFoundError: [Errno 2] No such file or directory: 'data/abundance.txt'

## peak_merger(list_masses, list_abundances)
This function takes the two lists outputted by main_function, and merges the values in list_abundances if their values in list_masses are the same.

In [7]:
import MASSiveChem.MASSiveChem as MC

output = MC.peak_merger([336.09, 334.094, 336.084, 334.088, 337.086, 335.087, 334.09, 335.095, 334.095, 334.097, 333.091, 334.09], 
                        [0.006155135615344779, 0.1389218494465703, 0.00025689925174544854, 0.005798234418254755, 0.00016431194999935186,
                        0.03458766547486355, 0.006161698124975695, 0.006260193058916763, 0.001252038611783352, 0.0019909461317594106, 
                        0.7806460744469201, 0.006245168595575361],
                        0.01)

print(output)

([335.092, 335.086, 333.71807812500003, 334.09], [0.14507698506191508, 0.006055133670000204, 0.8310629277992183, 0.006245168595575361])
